In [1]:
import os
import sys
import pickle
from inspect import signature

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics
from sklearn.preprocessing import normalize

get_dir = os.path.dirname
PROJ_ROOT = get_dir(get_dir(os.path.abspath('__file__')))
print(PROJ_ROOT)
sys.path.append(os.path.join(PROJ_ROOT, 'src'))

from data import dataset
from model import model 

C:\Users\48519\Professional Stuff\various\machine_learning\mouse_disambiguation


In [2]:
with open(os.path.join(dataset.DEFAULT_DATA_MODEL_DIRECTORY, 'data_model.pickle'), 'rb') as f:
    dat_mod = pickle.load(f)
    
class_params = model.DEFAULT_CLASSIFIER_SETTINGS

vectorizer_settings = dataset.DEFAULT_VECTORIZER_SETTINGS

pred_model = model.build_mmdisambiguator(
    data_model_params=vectorizer_settings, 
    data_model_path=os.path.join(dataset.DEFAULT_DATA_MODEL_DIRECTORY, 'data_model.pickle'),
    classificator_parameters=class_params
)

In [3]:
# train data
FEATURES_PATH = dataset.DEFAULT_FEATURES_DIRECTORY
TRAIN_FEATURES_PATH = os.path.join(FEATURES_PATH, 'train.npy')
data = np.load(TRAIN_FEATURES_PATH, allow_pickle=True)

In [4]:
# train the classifier from features and print report
features_trained_report = pred_model.train(data=data[:,:-1],classes=data[:,-1], report=True, source='features')
print(features_trained_report)

{'accuracy': 0.9953271028037384, 'precision': 0.9818181818181818, 'recall': 1.0, 'f1': 0.9908256880733944, 'confussion_matrix': [[54, 0], [1, 159]]}


In [5]:
# load train text data:
TEXT_PATH = dataset.DEFAULT_PROCESSED_TEXT_DATA_DIRECTORY
TRAIN_TEXT_PATH = os.path.join(TEXT_PATH, 'train.csv')
textdata = pd.read_csv(TRAIN_TEXT_PATH, sep=';')
textdata.head()

,text,class
0,The northern hopping mouse Notomys aquilo is f...,animal
1,Combination of left or rightclick and the mous...,device
2,Though not commonly referred to as optical mou...,device
3,The mouse was a simple optomechanical device a...,device
4,Common terms used to refer to different agessi...,animal


In [6]:
# train the classifier on text and expect identical results
text_trained_report = pred_model.train(data=textdata.iloc[:,0],classes=textdata.iloc[:,1], report=True, source='text')
print(text_trained_report)

[('device', 160), ('animal', 54)]
{'accuracy': 0.9953271028037384, 'precision': 0.9818181818181818, 'recall': 1.0, 'f1': 0.9908256880733944, 'confussion_matrix': [[54, 0], [1, 159]]}


In [9]:
# load validation features and text
VALIDATION_FEATURES_PATH = os.path.join(FEATURES_PATH, 'validation.npy')
VALIDATION_TEXT_PATH = os.path.join(dataset.DEFAULT_PROCESSED_TEXT_DATA_DIRECTORY, 'validation.csv')
validation = np.load(VALIDATION_FEATURES_PATH, allow_pickle=True)
validation_text = pd.read_csv(VALIDATION_TEXT_PATH, sep=';')


71


In [12]:
# predict on validation features and text and expect the same results
predict_on_features = pred_model.predict(validation[:,:-1])
predict_on_text = pred_model.predict(validation_text.iloc[:,0], source='text')

assert (predict_on_features[:,0] == predict_on_text[:,0]).all()

text before transofrmation: 71
text after transofrmation: 71


In [ ]:
predicted_valid = pred_model.predict(validaton[:,:-1], format='binary', mode='prediction')
print(validaton[:,:-1].sum(axis=1, keepdims=True))
print(validation_text.iloc[1,0])
to_print = 1
predicted_classes = pred_model.predict(validaton[:,:-1], format='text')[:,0]
summary = [(sentence, real, probs, pred) for sentence, real, probs, pred in zip(
    validation_text.iloc[:to_print,0], validation_text.iloc[:to_print,1],
    predicted_valid[:to_print], predicted_classes[:to_print])]
for row in summary:
    print(row)
pd.options.display.max_rows = 4000

# print(validation_text.iloc[:,1] != predicted_classes)
print(validation_text.iloc[[12, 24, 34, 35, 45, 50, 54, 57, 59, 62], 0])
print(predicted_valid[[12, 24, 34, 35, 45, 50, 54, 57, 59, 62]])


# print(validation_text.iloc[:to_print,0])